In [1]:
import pandas as pd
import numpy as np

def clean_data(df):

    df_to_clean = df.copy(deep=True)
    # Hay varias columnas que para la porción de ML no nos interesan. Las vamos a borrar
    try:
        df_to_clean.drop(columns=["location", "created_at", "updated_at", "property_url", "address", "zonaprop_code"]
                , inplace=True)
    except KeyError:
        df_to_clean.drop(columns=["location", "property_url", "address", "zonaprop_code"]
                , inplace=True)


    df_to_clean["price"] = df_to_clean["price"].str.replace(".000", "000").replace(".0", "")
    df_to_clean["price"] = df_to_clean["price"].str.replace(".0", "")

    # Convert extracted columns to numeric types
    df_to_clean[["total_area", "rooms", "bedrooms", "bathrooms", "garages", "price", "antiquity"]] = (
        df_to_clean[["total_area", "rooms", "bedrooms", "bathrooms", "garages", "price", "antiquity"]]
        .apply(pd.to_numeric, errors="coerce")
    )

    # Vamos a comenzar a llenar información faltante
    # Si la propiedad tiene NaN en garage, asumimos que no tiene
    df_to_clean.loc[df_to_clean["garages"].isna(), "garages"] = 0

    # Borramos las propiedades que no tienen precio
    df_to_clean = df_to_clean[~df_to_clean.price.isna()]

    # Borramos todas las propiedades que no tienen información de metros cuadrados
    df_to_clean = df_to_clean[~df_to_clean.total_area.isna()]

    # Vamos a trabajar ahora con numeros de cuartos, baños y ambientes
    # Pasamos las descripciones a lowercase para facilitar busquedas de strings
    df_to_clean["description"] = df_to_clean["description"].str.lower()

    # Si la propiedad menciona monoambiente en su descripción, asumimos que tiene 1 baño, 1 cuarto y 1 ambiente
    df_to_clean.loc[df_to_clean["description"].str.contains("monoambiente") & df_to_clean["bedrooms"].isna(), "bedrooms"] = 1
    df_to_clean.loc[df_to_clean["description"].str.contains("monoambiente") & df_to_clean["rooms"].isna(), "rooms"] = 1
    df_to_clean.loc[df_to_clean["description"].str.contains("monoambiente") & df_to_clean["bathrooms"].isna(), "bathrooms"] = 1

    # Vamos a seguir rellenando "rooms" en base a la descripción
    def rooms_filler(description):
        possible_rooms = [1,2,3,4,5,6,7,8,9,10]
        possible_descriptions = ["{} ambientes", "{} amb", "{} dormitorios", 
                                "{} dorm", "{} ambiente", "{}amb", "{} dor", "{}dorm", "{}  ambientes"]

        for i in possible_rooms:
            for j in possible_descriptions:
                if j.format(i) in description:
                    return i

    df_to_clean.loc[df_to_clean["rooms"].isna(), "rooms"] = df_to_clean.loc[df_to_clean["rooms"].isna(), "description"].apply(rooms_filler)

    # Eliminamos las propiedades que no tienen ambientes luego de este procesamiento
    df_to_clean = df_to_clean[~df_to_clean.rooms.isna()]

    # Realizamos conversores de baños y dormitorios

    # En resumen, si no tenemos cantidad de cuartos, asignamos la cantidad de cuartos menos 1
    # Sabemos que en este punto todos los registros tienen valor en rooms
    df_to_clean.loc[df_to_clean["bedrooms"].isna(), "bedrooms"] = df_to_clean.loc[df_to_clean["bedrooms"].isna()]["rooms"] - 1

    # En caso de que nos de 0, asumimos que es 1 ya que sería un monoambiente
    df_to_clean.loc[df_to_clean["bedrooms"] == 0, "bedrooms"] = 1

    def bathroom_converter(rooms):
        one_bathroom_values = [1,2,3]
        two_bathroom_values = [4,5,6]

        if rooms in one_bathroom_values:
            return 1
        elif rooms in two_bathroom_values:
            return 2
        else:
            return 3
        
    df_to_clean.loc[df_to_clean["bathrooms"].isna(), "bathrooms"] = df_to_clean.loc[df_to_clean["bathrooms"].isna()]["rooms"].apply(bathroom_converter)

    # Para antiguedad, si no tenemos valor, llenamos con el valor promedio del resto de las antiguedades
    # en el barrio
    df_to_clean['antiquity'] = df_to_clean['antiquity'].fillna(
        df_to_clean.groupby('neighborhood')['antiquity'].transform('mean')
    )

    # Round up to the nearest natural number
    df_to_clean['antiquity'] = np.ceil(df_to_clean['antiquity'])

    # Para muchos casos no tenemos antiguedad por promedio. Tomamos información de Internet y las rellenamos
    avg_antiquity = {
        "Recoleta": 50,  # Historical, many buildings from early 20th century
        "Núñez": 40,  # Mix of older houses and newer developments
        "Palermo Hollywood": 30,  # Many mid-century and newer constructions
        "Puerto Madero": 20,  # Mostly new developments since the 1990s
        "Centro / Microcentro": 70,  # Historic center with older buildings
        "Las Cañitas": 40,  # Trendy area with a mix of old and new
        "Palermo Soho": 40,  # Similar to Hollywood, slightly older buildings
        "Monte Castro": 50,  # Traditional residential area
        "Almagro Norte": 60,  # Older residential area
        "Tribunales": 80,  # Historic legal and business district
        "San Nicolás": 70,  # Similar to Microcentro
        "Monserrat": 80,  # One of the oldest neighborhoods
        "Belgrano R": 50,  # Residential, mix of old and newer homes
        "Palermo Nuevo": 30,  # Newer part of Palermo
        "Palermo Chico": 40,  # Upscale, many mid-century properties
        "Belgrano Chico": 40,  # Similar to Palermo Chico
        "Palermo Viejo": 50,  # Older buildings, many renovated
        "Retiro": 70,  # Historic with some modern developments
        "La Paternal": 50,  # Older residential neighborhood
        "Caballito Norte": 60,  # Older family homes
        "Belgrano C": 50,  # Similar to Belgrano R
        "Caballito Sur": 60,  # Same as Norte
        "Parque Rivadavia": 60,  # Older buildings near park
        "Villa Pueyrredón": 50,  # Traditional middle-class area
        "Floresta Sur": 60,  # Mix of old houses and mid-century buildings
        "Primera Junta": 60,  # Similar to Caballito
        "Cid Campeador": 60,  # Similar to surrounding areas
        "Constitución": 80,  # Old and densely built
        "Botánico": 40,  # Around the gardens, mix of styles
        "Lomas de Núñez": 30,  # Newer developments
        "Distrito Quartier": 20,  # Newly developed
        "Temperley": 70,  # Older suburb
        "Flores Sur": 60,  # Similar to Floresta
        "Almagro Sur": 60,  # Similar to Norte
        "Flores Norte": 60,  # Same as Sur
        "La Boca": 80,  # Historic with some modern projects
        "Parque Chas": 50,  # Traditional middle-class area
        "Floresta Norte": 60,  # Similar to Sur
        "Agronomía": 50,  # Near university, mix of styles
        "Otro": 50,  # Placeholder for undefined neighborhoods
        "Puerto Retiro": 70,  # Near historic Retiro
        "Barrio Parque": 40,  # Upscale, mid-century
        "Barrio Chino": 30,  # Newer commercial developments
        "Naón": 50,  # Mix of older homes
        "Parque Avellaneda": 60,  # Similar to Parque Chas
        "Catalinas": 20,  # Modern skyscrapers
        "Los Perales": 50,  # Traditional residential
        "Villa Riachuelo": 50,  # Outlying older area
        "Barrio Parque General Belgrano": 50  # Older homes, quieter
    }

    def antiquity_filler(neighborhood):
        return avg_antiquity.get(neighborhood, 50)

    df_to_clean.loc[df_to_clean["antiquity"].isna(), "antiquity"] = df_to_clean.loc[df_to_clean["antiquity"].isna()]["neighborhood"].apply(antiquity_filler)

    # Borramos la columna "expenses ya que no tenemos un buen uso por ahora"
    df_to_clean.drop(columns=["expenses"], inplace=True)

    # Borramos la descripción ya que no la vamos a usar en el modelo
    df_to_clean.drop(columns=["description"], inplace=True)

    # El precio tiene que ser mayor a 0 obligatoriamente
    print("Registros antes de borrar los precios menores a 1000: ", len(df_to_clean))
    df_to_clean = df_to_clean[df_to_clean["price"] > 1000]
    print("Registros después de borrar los precios menores a 1000: ", len(df_to_clean))

    # El área total debe ser mayor a 0 obligatoriamente
    print("Registros antes de borrar las superficies menores a 10: ", len(df_to_clean))
    df_to_clean = df_to_clean[df_to_clean["total_area"] > 10]
    print("Registros después de borrar las superficies menores a 10: ", len(df_to_clean))

    # Vemos que el precio tiene una distribución asimétrica a la derecha
    # Vamos a ahora quitar los outliers en relacion al precio
    print(f"Limpieza precio")

    # Calculamos los cuantiles
    Q1 = df_to_clean['price'].quantile(0.25)
    Q3 = df_to_clean['price'].quantile(0.75)
    IQR = Q3 - Q1

    # Calculamos los límites a partir de los cuantiles
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"Linea de borrado mínima': {lower_bound}")
    print(f"Linea de borrado máxima': {upper_bound}")

    # identificamos los outliers y luego los borramos
    outliers = df_to_clean[(df_to_clean['price'] < lower_bound) | (df_to_clean['price'] > upper_bound)]
    print(f"\nSe detectó la siguiente cantidad de outliers: {outliers.shape[0]}")

    # Remove outliers from the DataFrame
    df_to_clean = df_to_clean[(df_to_clean['price'] >= lower_bound) & (df_to_clean['price'] <= upper_bound)]

    print("\nNueva cantidad de datos luego de borrado:")
    print(df_to_clean.shape)

    # Tenemos un problema similar con la superficie total. Tomamos un approach similar

    # Vemos que el precio tiene una distribución asimétrica a la derecha
    # Vamos a ahora quitar los outliers en relacion al precio

    # Calculamos los cuantiles
    print(f"Limpieza superficie total")

    Q1 = df_to_clean['total_area'].quantile(0.25)
    Q3 = df_to_clean['total_area'].quantile(0.75)
    IQR = Q3 - Q1

    # Calculamos los límites a partir de los cuantiles
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"Linea de borrado mínima': {lower_bound}")
    print(f"Linea de borrado máxima': {upper_bound}")

    # identificamos los outliers y luego los borramos
    outliers = df_to_clean[(df_to_clean['total_area'] < lower_bound) | (df_to_clean['total_area'] > upper_bound)]
    print(f"\nSe detectó la siguiente cantidad de outliers: {outliers.shape[0]}")

    # Remove outliers from the DataFrame
    df_to_clean = df_to_clean[(df_to_clean['total_area'] >= lower_bound) & (df_to_clean['total_area'] <= upper_bound)]

    print("\nNueva cantidad de datos luego de borrado:")
    print(df_to_clean.shape)

    return df_to_clean

In [2]:
import pandas as pd
from dotenv import load_dotenv
import boto3
import os
from io import StringIO

# Debemos normalizar un poco la información antes de generar métricas.
# Además, definimos algunas constantes

BARRIO = "PALERMO"
AMBIENTES = 3
HISTORIC_FILE_PATH = "alquilerescaba_202501.xlsx"

load_dotenv()
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY_ID')
)
s3_client = session.client('s3')
BUCKET_NAME = os.getenv('BUCKET_NAME')

In [3]:
# Agarramos de zonaprop los dos archivos mas nuevos

def extract_date(file):
    date_part = file.split('_')[-1].replace('.csv', '')  # Get "04022025"
    return pd.to_datetime(date_part, format="%d%m%Y")  # Convert to datetime

files = s3_client.list_objects_v2(Bucket=BUCKET_NAME, Prefix="scrapping/ZonaProp/")
sub_files = list()
for file in files["Contents"]:
    if ("/ZonaProp/STG" in file["Key"]) and (".csv" in file["Key"]):
        sub_files.append(file["Key"])

sorted_files = sorted(sub_files, key=extract_date, reverse=True)
newest_files = sorted_files[:2]

newest_file = newest_files[0]
previous_file = newest_files[1]

response = s3_client.get_object(Bucket=BUCKET_NAME, Key=newest_file)
csv_data = response['Body'].read().decode('utf-8')  # Convert bytes to string
df_new = pd.read_csv(StringIO(csv_data))

response = s3_client.get_object(Bucket=BUCKET_NAME, Key=previous_file)
csv_data = response['Body'].read().decode('utf-8')  # Convert bytes to string
df_old = pd.read_csv(StringIO(csv_data))

In [4]:
# Debido a que no nos coinciden los barrios de la info histórica con los de ZonaProp, vamos a hacer un mapeo
# y normalización.

import pandas as pd
from difflib import get_close_matches

# Lista de valores permitidos
valid_neighborhoods = [
    'ALMAGRO', 'BALVANERA', 'BELGRANO', 'CABALLITO', 'COLEGIALES', 'DEVOTO',
    'FLORES', 'MONTSERRAT', 'NUNEZ', 'PALERMO', 'PARQUE PATRICIOS', 'PUERTO MADERO',
    'RECOLETA', 'RETIRO', 'SAN NICOLAS', 'SAN TELMO', 'VILLA CRESPO', 'VILLA DEL PARQUE', 'VILLA URQUIZA'
]

# Función para mapear los barrios
def map_neighborhood(neighborhood):
    neighborhood = neighborhood.upper()  # Convertimos a mayúsculas para estandarizar
    
    # Mapeos directos conocidos
    manual_mappings = {
        'MONSERRAT': 'MONTSERRAT',
        'NUÑEZ': 'NUNEZ',
        'CONGRESO': 'BALVANERA',
        'BARRIO NORTE': 'RECOLETA',
        'TRIBUNALES': 'SAN NICOLAS',
        'MICROCENTRO': 'SAN NICOLAS',
        'CENTRO / MICROCENTRO': 'SAN NICOLAS',
        'BARRACAS': 'PARQUE PATRICIOS',
        'CONSTITUCIÓN': 'SAN TELMO',
        'POMPEYA': 'PARQUE PATRICIOS',
        'MATADEROS': 'FLORES',
        'LINIERS': 'FLORES',
        'VERSALLES': 'VILLA URQUIZA',
        'VILLA SOLDATI': 'PARQUE PATRICIOS',
        'VILLA RIACHUELO': 'PARQUE PATRICIOS',
        'VILLA LUGANO': 'PARQUE PATRICIOS'
    }
    
    if neighborhood in manual_mappings:
        return manual_mappings[neighborhood]
    
    # Buscar coincidencias aproximadas
    match = get_close_matches(neighborhood, valid_neighborhoods, n=1, cutoff=0.6)
    return match[0] if match else 'OTRO'

# Aplicar la función de mapeo
df_old['normalized_neighborhood'] = df_old['neighborhood'].apply(map_neighborhood)
df_new['normalized_neighborhood'] = df_new['neighborhood'].apply(map_neighborhood)

# Dropeamos los que tienen valor OTRO
df_old = df_old[df_old['normalized_neighborhood'] != 'OTRO']
df_new = df_new[df_new['normalized_neighborhood'] != 'OTRO']

df_new

,property_url,zonaprop_code,price,expenses,address,location,description,neighborhood,total_area,rooms,bedrooms,bathrooms,garages,antiquity,normalized_neighborhood
0,https://www.zonaprop.com.ar/propiedades/clasif...,54051504,1708000.0,NaN,NaN,Capital Federal,Amplio 3 ambientes C/cochera nuevo A estrenar ...,Belgrano,70.0,3.0,2.0,2.0,1.0,NaN,BELGRANO
1,https://www.zonaprop.com.ar/propiedades/clasif...,55393779,550.000,100.000,NaN,Capital Federal,"Hermoso departamento de 2 ambientes al frente,...",Villa Santa Rita,38.0,2.0,1.0,1.0,NaN,NaN,VILLA URQUIZA
2,https://www.zonaprop.com.ar/propiedades/clasif...,55417978,3904000.0,NaN,NaN,Capital Federal,Excelente departamento en alquiler. Totalmente...,Recoleta,160.0,4.0,3.0,2.0,1.0,NaN,RECOLETA
3,https://www.zonaprop.com.ar/propiedades/clasif...,55568400,2928000.0,800.000,NaN,Palermo,"Palacio alcorta. Exclusivo loft, impecable est...",Palermo Chico,120.0,3.0,2.0,1.0,1.0,NaN,PALERMO
4,https://www.zonaprop.com.ar/propiedades/clasif...,55133119,8906000.0,NaN,NaN,Capital Federal,"Ubicación: Puerto Madero, la torre más exclusi...",Puerto Madero,249.0,4.0,3.0,5.0,2.0,NaN,PUERTO MADERO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12361,https://www.zonaprop.com.ar/propiedades/clasif...,55196163,600.000,160.000,NaN,Capital Federal,Rincon Y av. Rivadavia2 dormitorios 2 bañossem...,Congreso,NaN,3.0,2.0,1.0,NaN,NaN,BALVANERA
12362,https://www.zonaprop.com.ar/propiedades/clasif...,55036063,152.000,NaN,NaN,Capital Federal,"estas propiedades son ficticias, no contactar!",Colegiales,NaN,3.0,2.0,1.0,NaN,NaN,COLEGIALES
12363,https://www.zonaprop.com.ar/propiedades/clasif...,55455426,1.000.000,NaN,NaN,Capital Federal,Propiedades para soñar,Barrio Norte,NaN,NaN,NaN,NaN,NaN,NaN,RECOLETA
12364,https://www.zonaprop.com.ar/propiedades/clasif...,33479520,7.000,NaN,NaN,Capital Federal,Corredor Responsable: Cristian Arnal Ponti - C...,Recoleta,21.0,1.0,1.0,1.0,NaN,NaN,RECOLETA


In [5]:
df_new = clean_data(df_new)
df_new.head()

df_old = clean_data(df_old)
df_old.head()

Registros antes de borrar los precios menores a 1000:  10592
Registros después de borrar los precios menores a 1000:  10181
Registros antes de borrar las superficies menores a 10:  10181
Registros después de borrar las superficies menores a 10:  10162
Limpieza precio
Linea de borrado mínima': -300000.0
Linea de borrado máxima': 1700000.0

Se detectó la siguiente cantidad de outliers: 1516

Nueva cantidad de datos luego de borrado:
(8646, 9)
Limpieza superficie total
Linea de borrado mínima': 3.0
Linea de borrado máxima': 91.0

Se detectó la siguiente cantidad de outliers: 366

Nueva cantidad de datos luego de borrado:
(8280, 9)
Registros antes de borrar los precios menores a 1000:  10895
Registros después de borrar los precios menores a 1000:  10496
Registros antes de borrar las superficies menores a 10:  10496
Registros después de borrar las superficies menores a 10:  10474
Limpieza precio
Linea de borrado mínima': -225000.0
Linea de borrado máxima': 1575000.0

Se detectó la siguiente

,price,neighborhood,total_area,rooms,bedrooms,bathrooms,garages,antiquity,normalized_neighborhood
2,420000.0,Tribunales,44.0,2.0,1.0,1.0,0.0,80.0,SAN NICOLAS
5,320000.0,Barrio Norte,20.0,1.0,1.0,1.0,0.0,50.0,RECOLETA
6,650000.0,Belgrano,42.0,2.0,1.0,1.0,0.0,50.0,BELGRANO
9,500000.0,Recoleta,43.0,2.0,1.0,1.0,0.0,50.0,RECOLETA
10,450000.0,Centro / Microcentro,45.0,2.0,1.0,1.0,0.0,70.0,SAN NICOLAS


In [8]:
tmp = df_new.to_dict(orient="records")

In [10]:
import json
with open('outputfile.json', 'w') as fout:
    json.dump(tmp, fout)

In [2]:
import pandas as pd
import numpy as np

df_historic = pd.read_excel("workdir/alquilerescaba_202502.xlsx")
df_historic.head(2)

,X,Barrio,Inmueble,Mes,Mediana.a.precios.corrientes,Mediana.a.precios.constantes,Mediana.por.m2.a.precios.corrientes,Mediana.por.m2.a.precios.constantes
0,1,ALMAGRO,Departamento,2018-01,9500.0,9500.0,217.948718,217.948718
1,159,ALMAGRO,Dos o tres ambientes,2018-01,9500.0,9500.0,220.588235,220.588235


In [5]:
df_historic = df_historic[df_historic["Inmueble"] == "Departamento"]
df_historic.columns

Index(['X', 'Barrio', 'Inmueble', 'Mes', 'Mediana.a.precios.corrientes',
       'Mediana.a.precios.constantes', 'Mediana.por.m2.a.precios.corrientes',
       'Mediana.por.m2.a.precios.constantes'],
      dtype='object')

In [4]:
historic_data = df_historic.to_dict(orient="records")

import json
with open('historic.json', 'w') as fout:
    json.dump(historic_data, fout)